In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))

In [2]:
import vectorian

In [12]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import vectorian.alignment

class SlidingGapCostWidget:
    def __init__(self, description, construct, max=1.0):
        self._construct = construct
        
        self._cost = widgets.FloatSlider(
            value=0,
            min=0,
            max=max,
            step=0.01,
            description=description,
            disabled=False)

        self._plot = widgets.Image(
            value=b'',
            format='png',
            width=300,
            height=400,
        )

        self.update_plot()
        self._cost.observe(self.on_changed, names='value')
        
        self._vbox = widgets.VBox([self._cost, self._plot])
     
    def make(self):
        return self._construct(self._cost.value)
    
    def update_plot(self):
        fig, ax = plt.subplots(1, 1, figsize=(5, 2))
        im_data = self.make().plot_to_image(
            fig, ax, 20, format='png')
        plt.close()
        self._plot.value = im_data

    def on_changed(self, change):
        # cost = change.new
        self.update_plot()
        
    @property
    def widget(self):
        return self._vbox
    
class ConstantGapCostWidget(SlidingGapCostWidget):
    def __init__(self):
        super().__init__('Cost:', vectorian.alignment.ConstantGapCost)

class LinearGapCostWidget(SlidingGapCostWidget):
    def __init__(self):
        super().__init__('Cost:', vectorian.alignment.LinearGapCost)

class ExponentialGapCostWidget(SlidingGapCostWidget):
    def __init__(self):
        super().__init__('Cutoff:', vectorian.alignment.ExponentialGapCost, max=20)
        
class GapCostWidget:
    _types = [
        ('Constant', ConstantGapCostWidget),
        ('Linear', LinearGapCostWidget),
        ('Exponential', ExponentialGapCostWidget)
    ]
    
    def __init__(self):
        self._type = widgets.Dropdown(
            options=[x[0] for x in self._types],
            value='Linear',
            description='Gap Type:',
            disabled=False)
        
        self._fine_tune = LinearGapCostWidget()
        
        self._type.observe(self.on_changed, names='value')

        self._vbox = widgets.VBox([self._type, self._fine_tune.widget])

    def on_changed(self, change):
        i = [x[0] for x in self._types].index(change.new)
        self._fine_tune = self._types[i][1]()
        self._vbox.children = [self._type, self._fine_tune.widget]

    @property
    def widget(self):
        return self._vbox

        
w = GapCostWidget()
w.widget